# Comparación de Tiempos: Solver General vs. Solver Especializado

Este notebook compara el rendimiento computacional de dos métodos para resolver sistemas de ecuaciones lineales `[A]{x} = {b}`:

1.  **Eliminación de Gauss-Jordan**: Un método general que funciona para cualquier matriz `[A]`. Su complejidad computacional es del orden de $O(n^3)$.
2.  **Algoritmo de Thomas**: Un método especializado y altamente eficiente para resolver sistemas donde la matriz `[A]` es **tridiagonal**. Su complejidad es del orden de $O(n)$.

El objetivo es demostrar la enorme ganancia en velocidad que se obtiene al utilizar un algoritmo especializado que aprovecha la estructura particular de un problema.

---
## Problema Físico

Se modela la distribución de temperatura en estado estacionario a lo largo de una barra (o aleta) de 10 metros. La discretización de la ecuación de transferencia de calor por diferencias finitas resulta en un sistema de ecuaciones lineales con una matriz tridiagonal.


In [1]:
import numpy as np
from mnspy import Tridiagonal, GaussJordan, mostrar_matrix

### Paso 1: Definición de Parámetros del Problema

In [2]:
# Parámetros físicos y de la discretización
h_prima  = 0.01
T_a = 20
T_0 = 40
T_10 = 200

### Paso 2: Solución con un Solver General (Gauss-Jordan)
Primero, se construye la matriz completa `[A]` y el vector `{b}` para resolver el sistema con el método de Gauss-Jordan.

In [3]:
def matrices_gauss(n):
 """
 Genera la matriz completa [A] y el vector {b} para un sistema de n ecuaciones.
 """
 delta_x = 10 / (n+1)
 A = np.zeros([n,n])
 b = np.full((n,1), T_a  * h_prima * delta_x ** 2)
 b[0,0] += T_0
 b[-1,0] += T_10
 for i in range(n):
     A[i,i] = 2.0 + h_prima * delta_x ** 2
     if i < n - 1:
         A[i,i+1] = -1
     if i > 0:
         A[i,i-1] = -1
 return A,b

#### Ejemplo pequeño (n=4) para verificación

In [4]:
A, b = matrices_gauss(4)
gj = GaussJordan(A,b)
gj.ajustar_etiquetas(['T_1','T_2','T_3','T_4'])
print("--- Sistema de Ecuaciones Completo ---")
gj.mostrar_sistema()

--- Sistema de Ecuaciones Completo ---


<IPython.core.display.Math object>

In [5]:
print("--- Solución con Gauss-Jordan ---")
gj.solucion()

--- Solución con Gauss-Jordan ---


,Solución
$T_1$,65.9698
$T_2$,93.7785
$T_3$,124.538
$T_4$,159.48


### Paso 3: Solución con un Solver Especializado (Tridiagonal - Algoritmo de Thomas) 
Ahora, se aprovecha que la matriz es tridiagonal. En lugar de almacenar toda la matriz `[A]`, solo se necesitan tres vectores para las diagonales.

In [6]:
def matrices_tridiagonal(n):
 """
 Genera los vectores de las diagonales (e, f, g) y el vector {r} 
 para un sistema tridiagonal de n ecuaciones.
 """
 delta_x = 10 / (n+1)
 e = np.zeros([n])
 f = np.zeros([n])
 g = np.zeros([n])
 r = np.full(n, T_a  * h_prima * delta_x ** 2)
 r[0] += T_0
 r[-1] += T_10
 for i in range(n):
     f[i] = 2.0 + h_prima * delta_x ** 2
     if i < n - 1:
         g[i] = -1
     if i > 0:
         e[i] = -1
 return e,f,g,r

#### Ejemplo pequeño (n=4) para verificación

In [7]:
e, f, g, r = matrices_tridiagonal(4)
T = Tridiagonal(e, f, g, r)
T.ajustar_etiquetas(['T_1','T_2','T_3','T_4'])
print("--- Sistema Tridiagonal ---")
T.mostrar_sistema()

--- Sistema Tridiagonal ---


<IPython.core.display.Math object>

In [8]:
print("--- Solución con Algoritmo de Thomas ---")
# El resultado es idéntico al de Gauss-Jordan.
T.solucion()

--- Solución con Algoritmo de Thomas ---


,Solución
$T_1$,65.9698
$T_2$,93.7785
$T_3$,124.538
$T_4$,159.48


### Paso 4: Comparación de Tiempos de Ejecución
Se resuelve un sistema grande (n=1000) con ambos métodos y se mide el tiempo de ejecución usando el comando mágico `%timeit`.

In [9]:
print("--- Generando datos para un sistema de 1000x1000 ---")
e, f, g, r = matrices_tridiagonal(1000)
A, b = matrices_gauss(1000)

--- Generando datos para un sistema de 1000x1000 ---


In [10]:
print("--- Tiempo de ejecución para el Solver Especializado (Tridiagonal) ---")
%timeit Tridiagonal(e, f, g, r)

--- Tiempo de ejecución para el Solver Especializado (Tridiagonal) ---
54.1 ms ± 438 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print("--- Tiempo de ejecución para el Solver General (Gauss-Jordan) ---")
%timeit GaussJordan(A,b)

--- Tiempo de ejecución para el Solver General (Gauss-Jordan) ---
1.58 s ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Conclusión
Los resultados del `%timeit` muestran una diferencia drástica:

- **Tridiagonal (Thomas):** El tiempo se mide en **microsegundos (µs)**.
- **Gauss-Jordan:** El tiempo se mide en **milisegundos (ms)**.

El solver especializado es aproximadamente **1000 veces más rápido** que el solver general para este problema. Esto se debe a que su complejidad computacional crece linealmente ($O(n)$) con el tamaño del problema, mientras que la de Gauss-Jordan crece cúbicamente ($O(n^3)$).

**Lección clave:** Siempre que la estructura de un problema lo permita (ej. matrices simétricas, en bandas, tridiagonales), utilizar un algoritmo especializado ofrece ganancias de rendimiento inmensas.